### View f&uuml;r Spiel "Steine verschieben"
Zur Darstellung des Spielzustandes benutzen wir eine
MultiCanvas mit 3 Layern (Schachbrett, Steine von Spieler 0, Steine von Spieler 1).



Mit `view = View(game)` wird eine View f&uuml;r die Spielinstanz `game` erstellt.  
Die `__init__` Methode weist dem Attribut `game.callback` die Funktion  
`lambda event, data: self.move_stone(*data)` zu.  
Die Methode `move_stone(src, target, player)` l&ouml;scht dann den Spielstein an Position
`src` von der Leinwand und zeichnet in neu an Position `target`.

**Aufgabe**: Zeiche ein Schachbrett auf den Hintergrund-Layer
(Benutze z.B. die Funktion `draw_chessboard` aus dem Notebook `Canvas_Widget_I.ipynb` der Lektion 12).

In [ ]:
def draw_chessboard(canvas, color1 = 'grey', color2 = 'blue'):
    # zeichnet noch kein Schachbrett!
    canvas.fill_style = color1
    canvas.fill_rect(0, 0, canvas.width, canvas.height)

In [ ]:
from ipycanvas import MultiCanvas
class View:
    
    def __init__(self, game):
        self.game = game
        # callback registrieren
        game.callback = lambda event, data: self.move_stone(*data)
        
        self.mcanvas = MultiCanvas(3, width = 200, height = 200, 
                                   layout = {'border': '1px solid black'}
                                  )
        self.bg, self.l0, self.l1 = self.mcanvas
        self.layers = {'bg': self.bg, 0: self.l0, 1: self.l1}
        self.layers[0].fill_style = 'red'
        self.layers[1].fill_style = 'yellow'

        self.r = 10
        self.field_size = self.mcanvas.width / 8
        draw_chessboard(self.bg)
       
    def draw_position(self):
        for player, stones in self.game.position.items():
            for idx in stones:
                self.place_stone(player, idx)
            
    def place_stone(self, layer, idx):
        x, y = self.idx2pos(idx)
        self.layers[layer].fill_circle(x, y, self.r)
        
    def remove_stone(self, layer, idx):
        x, y = self.idx2pos(idx)
        self.layers[layer].clear_rect(x-self.r-0.5, y-self.r-0.5, 2*self.r+1)
        
    def move_stone(self, layer, src, target):
        self.remove_stone(layer, src)
        self.place_stone(layer, target)
        
    def idx2pos(self, idx):
        '''gibt x und y Koordinate der Mitte des Feldes mit Nummer idx zurueck'''
        row = idx // 8 
        col = idx % 8  
        return (col+0.5) * self.field_size, (row + 0.5)*self.field_size
    
    def _ipython_display_(self):
        display(self.mcanvas)

In [ ]:
# Klasse Game und Funktion engine importieren
from game_steine import Game, engine

In [ ]:
game = Game()
game.player_name[1] = engine
view = View(game)
view.draw_position()
view

In [ ]:
# Teste ob View auf Callback reagiert
game.move(0, 0, 9)
game

In [ ]:
game.move(0, 2, 11)
game

### Steine mit der Maus bewegen
Wir integrieren den Controller in der View.
Die Methoden `on_mouse_down` und `on_mouse_up` werden f&uuml;r die Events
Mausdown und Mausup registriert. Sie werden mit den Argumenten 
x und y-Koordinate der Klickposition aufgerufen.

- `on_mouse_down(x, y)`: berechnet Feldnummer und speicher diese im Attribut `selected_field`
- `on_mouse_up(x, y)`: l&ouml;scht den Spielstein auf dem geklickten Feld und zeichnet einen
neuen auf dem Feld, &uuml;ber dem die Maus losgelassen wurde.

Die als Callback f&uuml;r das Maus-Event `Maus-up` registrierte 
Methode `on_mouse_up` gibt den mit der Maus gemachten Zug an `game` weiter. Ist der Zug legal, wird er ausgef&uuml;hrt. 
`game` ruft dann das von der View registrierte Callback auf, welches den Zug auf der Leinwand darstellt.

In [ ]:
from ipycanvas import MultiCanvas

class View:
    
    def __init__(self, game):
        self.game = game
        # callback registrieren
        game.callback = lambda event, data: self.move_stone(*data)
        
        self.mcanvas = MultiCanvas(3, width = 200, height = 200, 
                                   layout = {'border': '1px solid black'}
                                  )
        self.bg, self.l0, self.l1 = self.mcanvas
        self.layers = {'bg': self.bg, 0: self.l0, 1: self.l1}
        self.layers[0].fill_style = 'red'
        self.layers[1].fill_style = 'yellow'

        self.r = 10
        self.field_size = self.mcanvas.width / 8
        draw_chessboard(self.bg)
        
        # Attribut fuer selektiertes Feld, Callbacks fuer Maus-Events registrieren
        self.selected_field = None
        self.mcanvas.on_mouse_down(self.on_mouse_down)
        self.mcanvas.on_mouse_up(self.on_mouse_up)
        
    def draw_position(self):
        for player, stones in self.game.position.items():
            for idx in stones:
                self.place_stone(player, idx)
            
    def place_stone(self, layer, idx):
        x, y = self.idx2pos(idx)
        self.layers[layer].fill_circle(x, y, self.r)
        
    def remove_stone(self, layer, idx):
        x, y = self.idx2pos(idx)
        self.layers[layer].clear_rect(x-self.r-0.5, y-self.r-0.5, 2*self.r+1)
        
    def move_stone(self, layer, src, target):
        self.remove_stone(layer, src)
        self.place_stone(layer, target)
        
    def idx2pos(self, idx):
        '''gibt x und y Koordinate der Mitte des Feldes mit Nummer idx zurueck'''
        row = idx // 8 
        col = idx % 8  
        return (col+0.5) * self.field_size, (row + 0.5)*self.field_size
      
    def pos2idx(self, pos):
        '''x,y Koordinaten der Klickposition und Feldnummer umrechnen'''
        x, y = pos
        col = int(x // self.field_size)
        row = int(y // self.field_size)
        return 8*row + col   
        
   
    def on_mouse_down(self, x, y):
        pos = (x, y)
        self.selected_field = self.pos2idx(pos)
        
    def on_mouse_up(self, x, y):   
        if self.selected_field is None:
            return
       
        pos = (x, y)
        target = self.pos2idx(pos)
        # Zug an Game-Instanz weitergeben
        self.game.move_stone(self.game.ptm, self.selected_field, target)  
        self.selected_field = None
        
    def _ipython_display_(self):
        display(self.mcanvas)

In [ ]:
game = Game()
game.player_name[1] = engine
view = View(game)
view.draw_position()
view

### Aufgabe
Obiger Code funktioniert leider nicht. Finde und korrigiere den Fehler.
1. Teste die Callbacks `view.on_mouse_down` und `view.on_mouse_up` indem du sieh mit sinnvollen Argumenten selber aufrufst, und
nachpr&uuml;fst, ob sie das gew&uuml;nschte tun.
1. Erstelle vor der Definition der Klasse `View` ein  Output-Widget und leite die Fehlermeldungen der Methoden `on_mouse_down` und `on_mouse_up` in dieses Widget um.  

```python
# Output Widget erstellen
from ipywidgets import Output
err_msg = Output(layout = {'border': '1px solid black'})

# alle relevante Methoden dekorieren
@err_msg.capture()
def on_mouse_down(self, x, y):   
    
# oder so (alter Output wird geloescht)    
@err_msg.capture(clear_output=True)
def on_mouse_down(self, x, y):   
```